<a href="https://colab.research.google.com/github/RudyMartin/dsai-2024/blob/main/MVPS/Camp-Rock-Paper-Scissors/team_GG_AI/21_37_baseline_gestures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DSAI Gestures Games AI Startup Project

Friday morning changes:


*   corrected missing initial_learning_rate and fine_tune_learning_rate
*   fixed h(history) type in evaluate model step
*   replaced rps with gesture classes in confusion matrix
*   replaced gestures with class_labels in several places


PLEASE RUN THIS WITH REAL DATA




In [ ]:
#!pip install tensorflow opencv-python

In [ ]:
### Config Project Folder
import os
from google.colab import drive
import datetime

# Record the start time for performance evaluation
start_time = datetime.datetime.now()

# Mount Google Drive
drive.flush_and_unmount()
!rm -rf /tmp/*

drive.mount("/content/gdrive", force_remount=True)
root_dir = "/content/gdrive/My Drive/dsai-2024/MVPS"
proj_dir = os.path.join(root_dir, 'Camp-Rock-Paper-Scissors')
os.chdir('/content/gdrive/My Drive/dsai-2024/MVPS/Camp-Rock-Paper-Scissors')
import os
from google.colab import drive
import datetime

# Record the start time for performance evaluation
start_time = datetime.datetime.now()

# Mount Google Drive
drive.flush_and_unmount()
!rm -rf /tmp/*

drive.mount("/content/gdrive", force_remount=True)
root_dir = "/content/gdrive/My Drive/dsai-2024/MVPS"
proj_dir = os.path.join(root_dir, 'Camp-Rock-Paper-Scissors')
os.chdir('/content/gdrive/My Drive/dsai-2024/MVPS/Camp-Rock-Paper-Scissors')

## PLEASE REPOINT THIS NEXT LINE TO YOUR DATA
rps_dir = os.path.join(proj_dir, 'gestures')  # this points to data folder
train_dir = os.path.join(rps_dir, 'train')
test_dir = os.path.join(rps_dir, 'test')
model_dir = os.path.join(rps_dir, 'model')
base_dir = rps_dir

# Create directories for train, test, and model activities
for dir in ['train', 'test', 'model']:
    os.makedirs(os.path.join(rps_dir, dir), exist_ok=True)

# Define class labels
class_labels = {
    0: "up",
    1: "down",
    2: "left",
    3: "right",
    4: "neutral"  # Add more labels as needed
}

# Create directories for each class in train and test
for label_name in class_labels.values():
    os.makedirs(os.path.join(train_dir, label_name.lower()), exist_ok=True)
    os.makedirs(os.path.join(test_dir, label_name.lower()), exist_ok=True)

# Record the end time and calculate the elapsed time
end_time = datetime.datetime.now()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time}")

# Look at the current directory
%ls

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/gdrive
Elapsed time: 0:00:32.565982
colab_setup/  functional_1_acc_graph.png   gestures/    model_tune/  rps/      rps_660/
data_prep/    functional_1_loss_graph.png  model_init/  README.MD    rps_114/


**Step 2: Set Up Data Collection**
We'll write a script to capture images from the webcam and save them into directories corresponding to the five directions: up, down, left, right, and neutral.

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
import os
from datetime import datetime

# Directory for saving images
# see config above

# Create directories for each gesture
# see config above

def get_unique_filename(directory, gesture, extension='jpg'):
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    base_filename = f"{gesture}_{timestamp}"
    filename = f"{base_filename}.{extension}"
    full_path = os.path.join(directory, gesture, filename)
    counter = 1
    while os.path.exists(full_path):
        filename = f"{base_filename}_{counter}.{extension}"
        full_path = os.path.join(directory, gesture, filename)
        counter += 1
    return full_path


split_dir = train_dir ##. change

# Capture and save images
for gesture in gestures:
    print(f"Capturing images for {gesture}. Press the 'Capture' button in the displayed UI.")
    for _ in range(2):  # Adjust the range for more images
        unique_filename = get_unique_filename(split_dir, gesture)
        filename = take_photo(unique_filename)
        print(f"Saved {filename}")

Capturing images for up. Press the 'Capture' button in the displayed UI.


<IPython.core.display.Javascript object>

Saved /content/gdrive/My Drive/dsai-2024/MVPS/Camp-Rock-Paper-Scissors/gestures/train/up/up_20240808_164017.jpg


<IPython.core.display.Javascript object>

Saved /content/gdrive/My Drive/dsai-2024/MVPS/Camp-Rock-Paper-Scissors/gestures/train/up/up_20240808_164029.jpg
Capturing images for down. Press the 'Capture' button in the displayed UI.


...

Saved /content/gdrive/My Drive/dsai-2024/MVPS/Camp-Rock-Paper-Scissors/gestures/train/right/right_20240808_164138.jpg
Capturing images for neutral. Press the 'Capture' button in the displayed UI.


<IPython.core.display.Javascript object>

Saved /content/gdrive/My Drive/dsai-2024/MVPS/Camp-Rock-Paper-Scissors/gestures/train/neutral/neutral_20240808_164145.jpg


<IPython.core.display.Javascript object>

Saved /content/gdrive/My Drive/dsai-2024/MVPS/Camp-Rock-Paper-Scissors/gestures/train/neutral/neutral_20240808_164157.jpg


In [ ]:
import glob

# Dictionary of class labels and corresponding directories
class_labels = {
    0: "up",
    1: "down",
    2: "left",
    3: "right",
    4: "neutral"  # Add more labels as needed
}

dirs = {
    'Train': train_dir,
    'Test': test_dir
}

# Nested loop to count images and print results
for dir_type, dir_path in dirs.items():
    print(f"{dir_type} directory: {dir_path}")
    for label_index, label_name in class_labels.items():
        num_images = len(glob.glob(f'{dir_path}/{label_name.lower()}/*.jpg'))
        print(f"Number of {dir_type.lower()} {label_name.lower()} images: {num_images}")

Train directory: /content/gdrive/My Drive/dsai-2024/MVPS/Camp-Rock-Paper-Scissors/gestures/train
Number of train up images: 50
Number of train down images: 50
Number of train left images: 50
Number of train right images: 50
Number of train neutral images: 50
Test directory: /content/gdrive/My Drive/dsai-2024/MVPS/Camp-Rock-Paper-Scissors/gestures/test
Number of test up images: 0
Number of test down images: 0
Number of test left images: 0
Number of test right images: 0
Number of test neutral images: 0


**Step 3: Prepare the Dataset**

After collecting enough images for each gesture, prepare the dataset for training.

The MobileNetV2 model expects input images to be 128x128 pixels with 3 color channels (RGB), but your training data has images of size 64x64 pixels. So set that in fthe function call.

In [ ]:
import os
import cv2
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Set seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Define class labels
class_labels = {
    0: "up",
    1: "down",
    2: "left",
    3: "right",
    4: "neutral"  # Add more labels as needed
}

def load_data(base_dir, img_size=(128, 128)):
    X = []
    y = []
    label_map = {label_name.lower(): i for i, label_name in class_labels.items()}

    for label_name in label_map.keys():
        gesture_dir = os.path.join(base_dir, label_name)
        for img_name in os.listdir(gesture_dir):
            img_path = os.path.join(gesture_dir, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)  # Read image in color mode
            img = cv2.resize(img, img_size)
            X.append(img)
            y.append(label_map[label_name])

    X = np.array(X, dtype='float32') / 255.0
    y = np.array(y, dtype='int')

    y = to_categorical(y, num_classes=len(class_labels))

    return train_test_split(X, y, test_size=0.2, random_state=seed)

# Load and split the data
X_train, X_test, y_train, y_test = load_data(train_dir)



In [ ]:
print(y_train.shape)

(200, 5)


**Step 4: Define, Train and Compile the Model**

Proceed with the model definition and training as usual. The seeds set earlier will ensure that the training process is reproducible.

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load MobileNetV2 with pre-trained weights, exclude top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Add custom layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)

# Define the model
model_name = "testeval_v001"
time_stamp = datetime.now().strftime('%Y%m%d_%H%M%S')
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base_model layers so they are not trained
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with an initial learning rate
initial_learning_rate = 0.001
model.compile(optimizer=Adam(learning_rate=initial_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=4, batch_size=32)

# Unfreeze the base_model layers and fine-tune the entire model
for layer in base_model.layers:
    layer.trainable = True

# Re-compile the model with a lower learning rate for fine-tuning
fine_tune_learning_rate = 0.0001
model.compile(optimizer=Adam(learning_rate=fine_tune_learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

# Configure callbacks for saving the best model and early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_file = f'best_{model_name}_{time_stamp}.keras'
model_checkpoint = ModelCheckpoint(os.path.join(model_dir, {model_file}), save_best_only=True)
# Train the model and get the history object
history = model.fit(X_train,
                    y_train,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stopping, model_checkpoint],
                    epochs=4,
                    batch_size=32)

# Save the model
#model_dir = 'override_model_directory'  # Specify your model directory if needed
#model_name = "testeval_v001"
#time_stamp = datetime.now().strftime('%Y%m%d_%H%M%S')
#model.save(f'{model_dir}/{model_name}_{time_stamp}.keras')

# OPTIONAL: Save the weights to file also
#time_stamp = datetime.now().strftime('%Y%m%d_%H%M%S')
#weights_file = f'{model_dir}/{model_name}_{time_stamp}_weights.keras'
#model.save_weights(weights_file)


Epoch 1/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 700ms/step - accuracy: 0.1580 - loss: 2.3496 - val_accuracy: 0.4600 - val_loss: 1.3315
Epoch 2/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 415ms/step - accuracy: 0.6394 - loss: 1.0637 - val_accuracy: 0.7200 - val_loss: 0.8692
Epoch 3/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 484ms/step - accuracy: 0.8774 - loss: 0.6349 - val_accuracy: 0.8400 - val_loss: 0.6128
Epoch 4/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 658ms/step - accuracy: 0.9342 - loss: 0.3931 - val_accuracy: 0.8600 - val_loss: 0.5153
Epoch 1/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.4610 - loss: 1.6419 - val_accuracy: 0.3200 - val_loss: 5.2201
Epoch 2/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.8884 - loss: 0.3555 - val_accuracy: 0.5800 - val_loss: 4.3676
Epoch 3/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.9454 - loss: 0.1880 - val_accuracy: 0.5400 - val_loss: 6.8514
Epoch 4/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 37s 4s/step - accuracy: 0.9927 - loss: 0.0410 - val_accuracy: 0.4800 - val_loss: 7.9202


PLEASE FIX THIS

**STEP 5: Evaluate the Model**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

# Define the evaluation function
def evaluate_model(generator, model):
    all_labels = []
    all_preds = []
    for images, labels in generator:
        preds = model.predict(images)
        all_preds.extend(preds)
        all_labels.extend(labels)
        if len(all_labels) >= len(generator.labels):  # Prevent infinite loop
            break
    y_pred = np.argmax(np.array(all_preds), axis=1)
    y_true = np.argmax(np.array(all_labels), axis=1)
    cm = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=generator.class_indices.keys(), yticklabels=generator.class_indices.keys())
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=generator.class_indices.keys()))

# Load your model (replace with your actual model path to override prior selection)
#model_name="mobilenetv2_head_gesture_model"
model1 = load_model(f'{model_dir}/{model_name}.keras')

# Define the test data generator
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_generator = test_datagen.flow_from_directory(
    f'{test_dir}',  # Adjust the path to your test data directory
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Ensure shuffle is set to False for proper evaluation
)

# Evaluate the model
evaluate_model(test_generator, model1)


ValueError: File not found: filepath=/content/gdrive/My Drive/dsai-2024/MVPS/Camp-Rock-Paper-Scissors/gestures/model/mobilenetv2_head_gesture_model.keras. Please ensure the file is an accessible `.keras` zip file.

In [ ]:
## 7. Evaluate Model

# helper functions
def show_scores1(model, h, X_train, y_train, X_test, y_test):
    loss, acc = model.evaluate(X_train, Y_train, verbose=0)
    print ("Training: accuracy   = %.6f loss = %.6f" % (acc, loss))
    loss, acc = model.evaluate(X_test, Y_test, verbose=0)
    print ("Validation: accuracy = %.6f loss = %.6f" % (acc, loss))
    if 'val_acc' in h.history:
        print ("Over fitting score   = %.6f" % over_fitting_score(h))
        print ("Under fitting score  = %.6f" % under_fitting_score(h))
    print ("Params count:", model.count_params())
    print ("stop epoch =", max(h.epoch))
    print ("nb_epoch =", h.params['epochs'])
    #print ("batch_size =", h.params['batch_size'])
    #print ("nb_sample =", h.params['samples'])
    view_acc1(h)
    id = model.name[-1]
    plt.savefig(model.name + '_acc_graph.png')
    plt.show()
    view_loss(h)
    plt.savefig(model.name + '_loss_graph.png')
    plt.show()

def view_acc1(h):
    # Accuracy history graph
    plt.plot(h.history['accuracy'])
    if 'val_accuracy' in h.history:
        plt.plot(h.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    leg = plt.legend(['train', 'validation'], loc='best')
    plt.setp(leg.get_lines(), linewidth=3.0)

def view_loss(h):
    # Loss history graph
    plt.plot(h.history['loss'])
    if 'val_loss' in h.history:
        plt.plot(h.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    leg = plt.legend(['train', 'validation'], loc='best')
    plt.setp(leg.get_lines(), linewidth=3.0)

 ## print both model accuracy and model loss by epoch
show_scores1(model1, history, X_train, y_train, X_test, y_test)

NameError: name 'h' is not defined

In [ ]:
## CONFUSION MATRIX** :  Contains True versus Predicted Vakues
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, f1_score, matthews_corrcoef, jaccard_score

# Define class labels
class_labels = {
    0: "up",
    1: "down",
    2: "left",
    3: "right",
    4: "neutral"  # Add more labels as needed
}

# Make predictions
predict_x = model1.predict(X_test)
y_pred = np.argmax(predict_x, axis=1)

# Convert numerical labels to class names
y_test_labels = [class_labels[int(y)] for y in y_test]
y_pred_labels = [class_labels[int(y)] for y in y_pred]

true_preds = [(x, y, p) for (x, y, p) in zip(X_test, y_test_labels, y_pred_labels) if y == p]
false_preds = [(x, y, p) for (x, y, p) in zip(X_test, y_test_labels, y_pred_labels) if y != p]

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
classes = np.array(list(class_labels.values()))

# Initialize the metrics dictionary
metrics = {
    'Class': [],
    'TP': [],
    'FN': [],
    'FP': [],
    'TN': [],
    'Recall': [],
    'Precision': [],
    'Accuracy': [],
    'F1 Score': [],
    'Specificity': [],
    'MCC': [],
    'Jaccard Index': []
}

# Calculate metrics for each class
for i, class_label in enumerate(classes):
    TP = conf_matrix[i, i]
    FN = conf_matrix[i, :].sum() - TP
    FP = conf_matrix[:, i].sum() - TP
    TN = conf_matrix.sum() - (TP + FN + FP)

    recall = TP / (TP + FN) if TP + FN != 0 else 0
    precision = TP / (TP + FP) if TP + FP != 0 else 0
    accuracy = (TP + TN) / (TP + TN + FP + FN) if TP + TN + FP + FN != 0 else 0
    f1 = f1_score(y_test_labels, y_pred_labels, labels=[class_label], average=None)[0]
    specificity = TN / (TN + FP) if TN + FP != 0 else 0
    mcc = matthews_corrcoef(y_test_labels, y_pred_labels)
    jaccard = jaccard_score(y_test_labels, y_pred_labels, labels=[class_label], average=None)[0]

    metrics['Class'].append(class_label)
    metrics['TP'].append(TP)
    metrics['FN'].append(FN)
    metrics['FP'].append(FP)
    metrics['TN'].append(TN)
    metrics['Recall'].append(recall)
    metrics['Precision'].append(precision)
    metrics['Accuracy'].append(accuracy)
    metrics['F1 Score'].append(f1)
    metrics['Specificity'].append(specificity)
    metrics['MCC'].append(mcc)
    metrics['Jaccard Index'].append(jaccard)

# Convert metrics dictionary to DataFrame
df_metrics = pd.DataFrame(metrics)

# Show the DataFrame with all metrics
print(df_metrics)

# Save the DataFrame to a CSV file
time_stamp = datetime.now().strftime('%Y%m%d_%H%M%S')
df_metrics.to_csv(f"df_metrics_{model_name}_{time_stamp}.csv", index=False)

ValueError: as_list() is not defined on an unknown TensorShape.

In [ ]:
## Unmount your drive when finished
drive.flush_and_unmount()